In [19]:
import os
import time
print(os.getcwd())

/mnt/c/Users/Komar/Lab/OutPutDir/LINE/GGCC


In [74]:
os.chdir('/mnt/c/Users/Komar/Lab/OutPutDir/LINE/AGCT/')
print(os.listdir())

['BC60', 'BC66', 'BC70', 'BLC1', 'CC107', 'CC127-2', 'CC18', 'CC46', 'CC5', 'CC82', 'CC94', 'CC96', 'CC99', 'GC39', 'GC41', 'LS1', 'LS6', 'M14', 'M21', 'M22', 'NC5', 'OC14', 'OC33', 'OC40', 'OC42', 'OC48', 'OC50', 'OC54', 'OT1', 'PC12', 'PC14', 'PC21', 'PC32', 'SkC6', 'StS1', 'UC2']


In [75]:
dirs = os.listdir() #[8:]
k = 0
avg_time = 0
for dirname in dirs:
    print(dirname, '---------------------------------------------------')
    curr_dir = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/AGCT/' + dirname
    if not os.path.isdir(curr_dir):
        continue
    os.chdir(curr_dir)
    DIR = os.getcwd() + '/nofiltration'
    t = time.time()
    # 1 Get somatic insertions
    file_names, df_somatic_list = get_somatic(curr_dir)
    # 2 Filtration
    writedown_filtered(DIR, file_names, df_somatic_list)
    # 3 UCSC BLAT parsing
    blat_parsing(DIR, file_names)
    # 4 BLAT analysis
    blat_analysis(DIR, file_names)
    # 5 RepeatMasker parsing
    blat_res, RM_res = get_10best_alings(DIR, file_names)
    RM_parsing(DIR, file_names, blat_res, RM_res)
    # 6 Merging
    merging(DIR, file_names)
    k += 1
    if k == 1:
        avg_time = int(time.time() - t)
    avg_time = (avg_time + int(time.time() - t)) / 2
    test = telegram_bot_sendtext(f"Postprocessing done for {dirname}_AGCT\n" + f"Duration: {int(time.time() - t)} sec\n" + 
                                 f"Time to finish (approx): {round(avg_time * (len(dirs) - k) / 3600, 1)} hours")     

BC60 ---------------------------------------------------
metatable shape: (1907, 32)
sample file names: ['BC60-L_S106_ME_L001__', 'MN-BC60-L_S108_ME_L001__']
null occurrences: 0
all not somatic: 828   including test: 0
somatic and quasi-somatic: (1079, 32)   including test: 0 

BC60-L_S106_ME_L001__ 441   test: 0
MN-BC60-L_S108_ME_L001__ 638   test: 0
overall num of somatic insertions:  1079 

BC60-L_S106_ME_L001__  was: 441  become: 36
BC60-L_S106_ME_L001__ reads were written
MN-BC60-L_S108_ME_L001__  was: 638  become: 71
MN-BC60-L_S108_ME_L001__ reads were written

Human BLAT Search


blat analysis:: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.18s/it]



before: 83
unique names: 23
after: 35 

before: 251
unique names: 35
after: 61 

Repeat Annotation Request Form


MN-BC60-L_S108_ME_L001__: 100%|█████████████████████████████████████████████████████████| 61/61 [10:22<00:00, 10.20s/it]



BC66 ---------------------------------------------------
metatable shape: (1927, 32)
sample file names: ['BC66-L_S102_ME_L001__', 'MN-BC66-L_S104_ME_L001__']
null occurrences: 0
all not somatic: 842   including test: 0
somatic and quasi-somatic: (1085, 32)   including test: 0 

BC66-L_S102_ME_L001__ 382   test: 0
MN-BC66-L_S104_ME_L001__ 703   test: 0
overall num of somatic insertions:  1085 

BC66-L_S102_ME_L001__  was: 382  become: 36
BC66-L_S102_ME_L001__ reads were written
MN-BC66-L_S104_ME_L001__  was: 703  become: 68
MN-BC66-L_S104_ME_L001__ reads were written

Human BLAT Search


...

StS1-L_S90_ME_L001__: 100%|█████████████████████████████████████████████████████████████| 68/68 [09:53<00:00,  8.73s/it]



UC2 ---------------------------------------------------
metatable shape: (1364, 32)
sample file names: ['MN-UC2-L_S124_ME_L001__', 'UC2-L_S122_ME_L001__']
null occurrences: 0
all not somatic: 604   including test: 0
somatic and quasi-somatic: (760, 32)   including test: 0 

MN-UC2-L_S124_ME_L001__ 254   test: 0
UC2-L_S122_ME_L001__ 506   test: 0
overall num of somatic insertions:  760 

MN-UC2-L_S124_ME_L001__  was: 254  become: 22
MN-UC2-L_S124_ME_L001__ reads were written
UC2-L_S122_ME_L001__  was: 506  become: 37
UC2-L_S122_ME_L001__ reads were written

Human BLAT Search


blat analysis:: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.87s/it]



before: 23
unique names: 11
after: 23 

before: 117
unique names: 24
after: 39 

Repeat Annotation Request Form


UC2-L_S122_ME_L001__: 100%|█████████████████████████████████████████████████████████████| 39/39 [05:25<00:00,  8.36s/it]


### Output data filtration

In [6]:
import pandas as pd
def get_somatic(path, show=True):
    # read pipeline output
    res = pd.read_table(path + '/metatable_ts.txt')
    if show:
        print('metatable shape:', res.shape)
    #print(res.head())

    # make list of file names
    file_names = []
    for col_name in res.columns:
        if col_name.find('_NUM_READS') != -1:
            file_names.append(col_name[:-10])
    if show:
        print('sample file names:', file_names)
    
    # checking for null occurrences 
    df_0 = res.copy(deep=True)
    for i in range(len(file_names)):
        df_0 = df_0[df_0[file_names[i] + '_NUM_READS'] == 0]
    if show:
        print('null occurrences:', df_0.shape[0])
    
    # not somatic (germline) insertions (exists in all samples)
    df_germ = res.copy(deep=True)
    for i in range(len(file_names)):
        df_germ = df_germ[df_germ[file_names[i] + '_NUM_READS'] > 0]
    if show:
        print('all not somatic:', df_germ.shape[0],  '  including test:', df_germ[df_germ['READNAME'].str.contains('ARTIFICIAL')].shape[0])
    
    # somatic and quasi-somatic (exists for example in 3/4 samples) insertions
    df_qsom = res.copy(deep=True)
    df_qsom = df_qsom.drop(df_germ.index, axis=0)
    if show:
        print('somatic and quasi-somatic:', df_qsom.shape, '  including test:', df_qsom[df_qsom['READNAME'].str.contains('ARTIFICIAL')].shape[0], '\n')
    
    # insertions specific only for this tissue/sample(file) 
    df_somatic_list = []
    overall_num = 0
    for i in range(len(file_names)):
        df = df_qsom.copy(deep=True)
        df = df[df[file_names[i] + '_NUM_READS'] > 0]
        for j in range(1, len(file_names)):
            df = df[df[file_names[i - j] + '_NUM_READS'] == 0]
        if show:
            print(file_names[i], df.shape[0], '  test:', df[df['READNAME'].str.contains('ARTIFICIAL')].shape[0])
        overall_num += df.shape[0]
        df_somatic_list.append(df.copy(deep=True))
    if show:
        print('overall num of somatic insertions: ', overall_num, '\n')
    return file_names, df_somatic_list

In [55]:
# preparing for copying into blat and filtering 
def writedown_filtered(DIR, file_names, df_somatic_list):
    if not os.path.exists(DIR + '/filtred'):
        os.makedirs(DIR + '/filtred')
    
    for i in range(len(file_names)):
        # filtration
        df_tmp = df_somatic_list[i].copy(deep=True)
        #df_tmp = df_tmp[df_tmp[file_names[i] + '_NUM_READS'] > 1].copy(deep=True)
        df_tmp = df_tmp[df_tmp['R1'].str.len() > 25].copy(deep=True)  # because Blat can aling seq > 25nt
        df_tmp = df_tmp[df_tmp['R2'].str.len() > 25].copy(deep=True)
        df_tmp = df_tmp[df_tmp['MDMATCH'] < (52+6+14 - 5)].copy(deep=True)  # template_switch_filtration: 12 primer + 6 dALU + 14 flank = 32
        print(file_names[i], ' was:', df_somatic_list[i].shape[0], ' become:', df_tmp.shape[0])
    
        df_tmp.to_csv(DIR + '/filtred/' + f'{file_names[i]}_filtered.csv', index=False)
        print(f'{file_names[i]} reads were written')
    print()

### UCSC Blat parsing

In [4]:
def complement(char):
    if char == 'A':
        return 'T'
    if char == 'T':
        return 'A'
    if char == 'C':
        return 'G'
    if char == 'G':
        return 'C'
    if char == 'N':
        return 'N'

def reverse_comp(seq):
    seq_ls = list(seq)
    new_seq = []
    for i in reversed(seq_ls):
        new_seq.append(complement(i))
    return "".join(new_seq)

In [3]:
def get_reads_fa(reads_list, reads_names, R_n=1):
    line = ''
    for read, name in zip(reads_list, reads_names):
        # if R_n == 1:  # for Alu 5' end
        if R_n == 2:  # for LINE 3' end
            line = line + '>' + name + f'_R{R_n}' + '\n' + reverse_comp(read) + '\n'  # because if insertion is "+" then R1 is rev_comp of + strand dna
        else:
            line = line + '>' + name + f'_R{R_n}' + '\n' + read + '\n'
    return line

In [5]:
def get_blat_output(driver, filtered_flanks, i, k, j, length=10):
    # set flank coords and submit
    range_input = driver.find_element(By.NAME, "userSeq")
    range_input.clear()
    #push 10 reads at once
    read_list = filtered_flanks[i][f'R{k}'].tolist()[j*10 : j*10 + length]
    name_list = filtered_flanks[i]['READNAME'].tolist()[j*10 : j*10 + length]
    range_input.send_keys(get_reads_fa(read_list, name_list, R_n=k))
    
    button = driver.find_element(By.NAME, "Submit")
    button.click()
    # get results 
    blat_out = ''
    try:
        #print('waiting')
        element = WebDriverWait(driver, 60).until(
                            EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                            )
        blat_out = element.text
    except:
        print(f'exception for {file_names[i]}')
    # get back to request form       
    driver.back()
    return blat_out

In [6]:
import pandas as pd
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

def blat_parsing(DIR, file_names):
    service = Service(executable_path="/usr/bin/chromedriver")
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    filtered_flanks = []
    for i in range(len(file_names)):
        df_tmp = pd.read_csv(DIR + '/filtred/' + f'{file_names[i]}_filtered.csv')
        filtered_flanks.append(df_tmp.copy(deep=True))
    
    # go to the UCSC Blat
    driver.get("https://genome.ucsc.edu/cgi-bin/hgBlat?hgsid=1711647570_KRW9QfFHAHaje8cp8sWT4EtpGupE&command=start")
    print(driver.title)
    
    # set Hg38
    selects = driver.find_elements(By.TAG_NAME, "select")
    for i in selects:
        if i.get_property("name") == "db":
            element = i
            break
    dropdown = Select(element)
    dropdown.select_by_visible_text("Dec. 2013 (GRCh38/hg38)")
    
    if not os.path.exists(DIR + '/blat_output'):
        os.makedirs(DIR + '/blat_output')
    
    for i in range(len(filtered_flanks)):
        for k in [1, 2]:
            output = ''
            for j in trange(int(filtered_flanks[i].shape[0] / 10), desc=f'{file_names[i]}_R{k}'):  
                output = output + '\n' + get_blat_output(driver, filtered_flanks, i, k, j)
            if filtered_flanks[i].shape[0] % 10 != 0:
                output = output + '\n' + get_blat_output(driver, filtered_flanks, i, k, int(filtered_flanks[i].shape[0] / 10), 
                                                         length=filtered_flanks[i].shape[0] % 10)
                
            with open(DIR + '/blat_output/' + f'{file_names[i]}_R{k}_blat_out_tmp.txt', 'w') as f:
                f.write(output)
                #print('res written')
    
    driver.quit()

    for i in range(len(filtered_flanks)):
        for k in [1, 2]:
            blat_out = pd.DataFrame(columns=['QUERY', 'SCORE', 'QSTART', 'QEND', 'QSIZE', 'IDENTITY', 'CHROM', 'STRAND', 'START', 'END', 'SPAN'])
            with open(DIR + '/blat_output/' + f'{file_names[i]}_R{k}_blat_out_tmp.txt', 'r') as f:
                output = f.read()
            output = output.split('\n')
            for line in output:
                    if line.count('browser'):
                        tmp = line.split(' ')
                        tmp = ' '.join(tmp).split()
                        blat_out.loc[len(blat_out.index)] = [tmp[2], tmp[3], tmp[4], tmp[5], tmp[6], tmp[7], tmp[8], tmp[9], tmp[10], tmp[11], tmp[12]]
            blat_out.to_csv(DIR + '/blat_output/' + f'{file_names[i]}_R{k}_blat_out.csv', index=False) 

### Blat analysis

In [35]:
import pandas as pd
from tqdm import trange

def blat_merge_and_filtration(filename):

    FRAGMENT_LEN = 1000  # from electroforeses flanks will be from ~60 to ~585

    #chomosome list
    chromosomes = list(range(1, 23))
    chromosomes.append('X')
    chromosomes.append('Y')
    chromosomes = ['chr' + str(x) for x in chromosomes]
    
    df1 = pd.read_table(filename + '_R1_blat_out.csv', sep=',')
    #df1 = df1.drop(['Unnamed: 0'], axis=1)
    
    df2 = pd.read_table(filename + '_R2_blat_out.csv', sep=',')
    #df2 = df2.drop(['Unnamed: 0'], axis=1)
    
    read_names = df1['QUERY'].unique()
    merget_df = pd.DataFrame(columns=['QUERY', 
                                      'SCORE_1', 'QSIZE_1', 'QSTART_1', 'QEND_1', 'IDENTITY_1', 'CHROM_1', 'STRAND_1', 'START_1', 'END_1', 'SPAN_1',
                                      'SCORE_2', 'QSIZE_2', 'QSTART_2', 'QEND_2', 'IDENTITY_2', 'CHROM_2', 'STRAND_2', 'START_2', 'END_2', 'SPAN_2'
                                     ]
                            )
    for name in read_names:
        df1_1 = df1[df1['QUERY'].str.contains(name)].copy(deep=True)
        df2_1 = df2[df2['QUERY'].str.contains(name[:-3])].copy(deep=True)
        for i, row_1 in df1_1.iterrows():
            tmp_df = df2_1[df2_1['CHROM'] == row_1['CHROM']]
            for i, row_tmp in tmp_df.iterrows():
                if abs(row_tmp['START'] - row_1['START']) <= FRAGMENT_LEN and abs(row_tmp['END'] - row_1['END']) <= FRAGMENT_LEN:
                    merget_df.loc[len(merget_df.index)] = [row_1['QUERY'][:-3], 
                                                           row_1['SCORE'], row_1['QSIZE'], row_1['QSTART'], row_1['QEND'], row_1['IDENTITY'],
                                                           row_1['CHROM'], row_1['STRAND'], row_1['START'], row_1['END'], row_1['SPAN'],
                                                           row_tmp['SCORE'], row_tmp['QSIZE'], row_tmp['QSTART'], row_tmp['QEND'], row_tmp['IDENTITY'], 
                                                           row_tmp['CHROM'], row_tmp['STRAND'], row_tmp['START'], row_tmp['END'], row_tmp['SPAN'] 
                                                          ]
    #print(name[:-3], df1_1.shape[0], df2_1.shape[0], merget_df.shape[0])
    
    #drop alt chr
    merget_df.drop(merget_df[ ~merget_df['CHROM_1'].isin(chromosomes) ].index, axis=0, inplace=True)
    #drop matches with different directions 
    merget_df = merget_df[merget_df['STRAND_1'] == merget_df['STRAND_2']].copy(deep=True)

    # get only high score
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_1'] > 0.9 * merget_df['QSIZE_1']].copy(deep=True)
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_2'] > 0.9 * merget_df['QSIZE_2']].copy(deep=True)

    # drop aligns which are not close to RE 5'end 
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='+') & (merget_df['QEND_1'] <= merget_df['QSIZE_1'] - 3) ].index, axis=0, inplace=True)   
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='-') & (merget_df['QSTART_1'] >= 3) ].index, axis=0, inplace=True)

    # drop strange aligns: "+" R2_start >= R1_end; "-" R1_start >= R2_end
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='+') & (merget_df['START_2'] >= merget_df['END_1']) ].index, axis=0, inplace=True)   
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='-') & (merget_df['START_1'] >= merget_df['END_2']) ].index, axis=0, inplace=True)
        
    # calc flank metrics and insertion coord 
    if merget_df.shape[0] > 0:
        for i, row in merget_df.iterrows():
            merget_df.loc[i, 'F_CHR'] = row['CHROM_1']
            start = int(min(row['START_1'], row['START_2'], row['END_1'], row['END_2']))
            end = int(max(row['START_1'], row['START_2'], row['END_1'], row['END_2']))
            merget_df.loc[i, 'F_START'] = start
            merget_df.loc[i, 'F_END'] = end
            merget_df.loc[i, 'F_LEN'] = int(end - start) + 1
            if row['STRAND_1'] == '+':
                merget_df.loc[i, 'INS_COORD'] = end
            else:
                merget_df.loc[i, 'INS_COORD'] = start - 1
            
    return merget_df.copy(deep=True)

In [8]:
def blat_analysis(DIR, file_names):
    if not os.path.exists(DIR + '/blat_filtred'):
        os.makedirs(DIR + '/blat_filtred')
    
    for i in trange(len(file_names), desc='blat analysis:'):
        df_tmp = blat_merge_and_filtration(DIR + '/blat_output/' + file_names[i])
        df_tmp.to_csv(DIR + '/blat_filtred/' + f'{file_names[i]}_blat_filtred.csv', index=False)
    print()

### RepeatMasker

In [9]:
# get 5 best aligns for each read
import pandas as pd

def get_10best_alings(DIR, file_names):
    blat_res = []
    RM_res = []
    for name in file_names:
        blat_res.append(pd.read_table(DIR + '/blat_filtred/' + f'{name}_blat_filtred.csv', sep=','))
        RM_res.append(pd.DataFrame(columns=['QUERY', 'EXTENSION', 'INS_CHR_B', 'INS_COORD_B', 'INS_STRAND_B', 'REPEAT_NAME', 'R_STRAND', 'R_CHR', 
                                            'R_START', 'R_END', 'INS_OVERLAP', 'DESC', 'F_LEN', 'R_LEN', 'F_START', 'F_END',
                                            'SCORE_1', 'QSIZE_1', 'SCORE_2', 'QSIZE_2', 'CHR_1', 'START_1', 'END_1', 'CHR_2', 'START_2', 'END_2', 
                                            ]))

    for blat_res_part in blat_res:
        print('before:', blat_res_part.shape[0])
        amount_list = blat_res_part.groupby(['QUERY']).count().sort_values(by=['QUERY'])['SCORE_1'].tolist()
        read_names = blat_res_part['QUERY'].unique()
        read_names.sort()
        print('unique names:', len(read_names))
        for num, name in zip(amount_list, read_names):
            if num > 5:
                indexes = blat_res_part[blat_res_part['QUERY'].str.contains(name)].index.tolist()[5:]
                blat_res_part.drop(indexes, axis=0, inplace=True)
                
        print('after:', blat_res_part.shape[0], '\n')

    return blat_res, RM_res

In [10]:
# return distanse from the insertion to the repeat nearest end and description 
# if they overlap - positive number, else - negative
def calc_overlap(ins_coord, r_start, r_end):
    ins_coord = int(ins_coord)
    r_start = int(r_start)
    r_end = int(r_end)
    if r_start < ins_coord:
        if r_end < ins_coord:
            return r_end - ins_coord, 'ins after the repeat'  # negative - insertion after the repeat
        else:
            return min(r_end - ins_coord, ins_coord - r_start), 'ins in the repeat'  # positive - insertion in the repeat
    else:
        # r_start >= ins_coord
        return ins_coord - r_start, 'ins before the repeat'  # negative - insertion before the repeat

In [11]:
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

def RM_parsing(DIR, file_names, blat_res, RM_res):
    service = Service(executable_path="/usr/bin/chromedriver")
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # go to the RepeatMasker
    driver.get("https://www.repeatmasker.org/cgi-bin/AnnotationRequest")
    print(driver.title)
    
    # set Hg38
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "genome"))
        )
        dropdown = Select(element)
        dropdown.select_by_visible_text("Human - Dec 2013 - hg38")
    except:
        print("can't select genome")
    
    
    if not os.path.exists(DIR + '/RepeatMasker'):
        os.makedirs(DIR + '/RepeatMasker')
    
    # flank coords annotating
    j = 0
    EXTENSION = 20
    for blat_res_part, RM_res_part in zip(blat_res, RM_res):
        blat_res_part = blat_res_part.reset_index()
        for i in trange(blat_res_part.shape[0], desc=f'{file_names[j]}'):
            # set flank coords and submit
            try:
                range_input = WebDriverWait(driver, 15).until(
                              EC.presence_of_element_located((By.NAME, "range")) 
                              )
                #driver.find_element(By.NAME, "range")
                range_input.clear()
    
                low_coord = int(blat_res_part.loc[i, 'INS_COORD'] - EXTENSION)
                high_coord = int(blat_res_part.loc[i, 'INS_COORD'] + EXTENSION)
                chr = blat_res_part.loc[i, 'F_CHR']
                coords = f"{chr}:{low_coord}-{high_coord}" 
                #print(coords)
                range_input.send_keys(coords)
               
                button = driver.find_element(By.NAME, ".submit")
                button.click()
            except:
                print("can't pass coords", blat_res_part.loc[i, 'QUERY'])
        
            # get results 
            try:
                element = WebDriverWait(driver, 15).until(
                          EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                          )
                #link = element.find_element(By.TAG_NAME, "a").get_attribute("href")
                output = element.text.split('\n')[2:]
                #print(output)
                for line in output:
                    tmp_line = line.split(' ')
                    tmp_line = ' '.join(tmp_line).split()
                    #print(tmp_line)
                    overlapping = calc_overlap(blat_res_part.loc[i, 'INS_COORD'], tmp_line[5], tmp_line[6])
                    #print(overlapping)
                    RM_res_part.loc[len(RM_res_part.index)] = [blat_res_part.loc[i, 'QUERY'], EXTENSION, blat_res_part.loc[i, 'F_CHR'],
                                                               blat_res_part.loc[i, 'INS_COORD'], blat_res_part.loc[i, 'STRAND_1'], tmp_line[9], 
                                                               tmp_line[8], tmp_line[4], tmp_line[5], tmp_line[6], overlapping[0], overlapping[1], 
                                                               blat_res_part.loc[i, 'F_LEN'], int(tmp_line[6]) - int(tmp_line[5]) + 1, 
                                                               blat_res_part.loc[i, 'F_START'], blat_res_part.loc[i, 'F_END'],
                                                               blat_res_part.loc[i, 'SCORE_1'], blat_res_part.loc[i, 'QSIZE_1'], 
                                                               blat_res_part.loc[i, 'SCORE_2'], blat_res_part.loc[i, 'QSIZE_2'],
                                                               blat_res_part.loc[i, 'CHROM_1'], blat_res_part.loc[i, 'START_1'], 
                                                               blat_res_part.loc[i, 'END_1'], blat_res_part.loc[i, 'CHROM_2'], 
                                                               blat_res_part.loc[i, 'START_2'], blat_res_part.loc[i, 'END_2'],
                                                              ]
            except:
                #print('no matching')
                RM_res_part.loc[len(RM_res_part.index)] = [blat_res_part.loc[i, 'QUERY'], EXTENSION, blat_res_part.loc[i, 'F_CHR'],
                                                           blat_res_part.loc[i, 'INS_COORD'], blat_res_part.loc[i, 'STRAND_1'], '-', '-', 
                                                           '-', 0, 0, 0, 'no repeat', blat_res_part.loc[i, 'F_LEN'],
                                                           0, blat_res_part.loc[i, 'F_START'], blat_res_part.loc[i, 'F_END'],
                                                           blat_res_part.loc[i, 'SCORE_1'], blat_res_part.loc[i, 'QSIZE_1'], 
                                                           blat_res_part.loc[i, 'SCORE_2'], blat_res_part.loc[i, 'QSIZE_2'],
                                                           blat_res_part.loc[i, 'CHROM_1'], blat_res_part.loc[i, 'START_1'], 
                                                           blat_res_part.loc[i, 'END_1'], blat_res_part.loc[i, 'CHROM_2'], 
                                                           blat_res_part.loc[i, 'START_2'], blat_res_part.loc[i, 'END_2'],
                                                          ]
            # get back to request form
            driver.back()
        RM_res_part.to_csv(DIR + '/RepeatMasker/' + f'{file_names[j]}_RM_res.csv', index=False)
        j += 1
    
    #quit from site
    driver.quit()
    print()

### Merging with pipeline info

In [12]:
# merging with pipeline info
def attach_pipeline_info(filename, df_res_part):
    filtered_flank = pd.read_csv(DIR + '/filtred/' + f'{filename}_filtered.csv')
    
    right_df = filtered_flank[['READNAME', 'CHR', 'POS', 'INS_STRAND', f'{filename}_NUM_READS', 'R1', 'R2']].copy(deep=True)
    left_df = df_res_part.copy(deep=True)
    
    return left_df.merge(right_df, left_on = 'QUERY', right_on = 'READNAME').drop(columns='READNAME')

In [13]:
def hamming_dist(x1, x2):
    j = 0
    for i in range(len(x1)):
        if x1[i] != x2[i]:
            j = j + 1
    return j

# return length of overlapping part
def find_overlap_len(str1, str2):
    if len(str1) < len(str2):
        str_tmp = str1
        str1 = str2
        str2 = str_tmp
    if len(str1) >= len(str2):
        delta = len(str1) - len(str2)
        for i in range(len(str1)):
            if delta - i >= 0:
                part1 = str1[i:len(str2) + i]
                part2 = str2
            else:
                part1 = str1[i:]
                part2 = str2[0:len(str2) + delta - i]
            # print(part1, part2)
            if hamming_dist(part1, part2) == 0:
                #print(part1, len(part1))
                return len(part1)
        return 0

In [14]:
# merging with pipeline info
# reads overlap calculation
def merging(DIR, file_names):
    RM_res_list = []
    i = 0
    for name in file_names:
        RM_res_list.append(pd.read_table(DIR + '/RepeatMasker/' + f'{name}_RM_res.csv', sep=','))
        RM_res_list[i] = attach_pipeline_info(name, RM_res_list[i])
        for j, row in RM_res_list[i].iterrows():
            RM_res_list[i].loc[j, 'READS_OVERLAP'] = find_overlap_len(row['R1'], reverse_comp(row['R2']))
        RM_res_list[i].to_csv(DIR + '/RepeatMasker/' + f'{name}_RM_res_merged.csv', index=False)
        i += 1

### Other

In [71]:
import requests

def text_replace(text):
    if len(text) == 0:
        return ' '
    text = text.replace('_', '\_')
    text = text.replace('*', '\*')
    text = text.replace('[', '\[')
    text = text.replace(']', '\]')
    text = text.replace('(', '\(')
    text = text.replace(')', '\)')
    text = text.replace('~', '\~')
    text = text.replace('`', '\`')
    text = text.replace('>', '\>')
    text = text.replace('#', '\#')
    text = text.replace('+', '\+')
    text = text.replace('-', '\-')
    text = text.replace('=', '\=')
    text = text.replace('|', '\|')
    text = text.replace('{', '\{')
    text = text.replace('}', '\}')
    text = text.replace('!', '\!')
    return text

def telegram_bot_sendtext(bot_message):
    bot_message = text_replace(bot_message)
    BOT_TOKEN = "6608136916:AAHB_1JsP8lsFLD7Uh4WoiajrUFCZToQ2cQ"
    admin_id = str(360072308)
    send_text = 'https://api.telegram.org/bot' + BOT_TOKEN + '/sendMessage?chat_id=' + admin_id + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()


### Drop questionable insertions 

In [4]:
import pandas as pd
import re
import os

def is_questionable(df_row, is_line):
    # no repeat - ok
    if df_row['DESC'] == 'no repeat':
        return False
    if is_line:
        # LINE
        # ins in the repeat
        if df_row['DESC'] == 'ins in the repeat':
            # repeat is in L1HS, <=L1PA7 => drop
            if df_row['REPEAT_NAME'] == 'L1HS' or \
               ( bool(re.search('L1PA[1-7]', df_row['REPEAT_NAME'])) and (not bool(re.search('L1PA[1-9][0-9]', df_row['REPEAT_NAME']))) ):
                return True
        # ins near the repeat
        else:
            # near <=L1PA7 or L1HS in 20nt => drop
            if ( df_row['REPEAT_NAME'] == 'L1HS' or \
                (bool(re.search('L1PA[1-7]', df_row['REPEAT_NAME'])) and (not bool(re.search('L1PA[1-9][0-9]', df_row['REPEAT_NAME'])))) \
               ) and abs(df_row['INS_OVERLAP']) <= 20:
                return True
    else:
        # Alu
        # ins in the repeat
        if df_row['DESC'] == 'ins in the repeat':
            # repeat is in L1HS, <=L1PA7 or Alu => drop
            if df_row['REPEAT_NAME'] == 'L1HS' or \
               ( bool(re.search('L1PA[1-7]', df_row['REPEAT_NAME'])) and (not bool(re.search('L1PA[1-9][0-9]', df_row['REPEAT_NAME']))) ) or \
               bool(re.search('Alu', df_row['REPEAT_NAME'])):
                return True
        # ins near the repeat
        else:
            # near Alu in 5nt => drop
            if bool(re.search('Alu', df_row['REPEAT_NAME'])) and abs(df_row['INS_OVERLAP']) <= 5:
                return True
    return False



def drop_questionable(DIR, file_names, is_line):
    
    for name in file_names:
        df = pd.read_table(DIR + '/RepeatMasker/' + f'{name}_RM_res_merged.csv', sep=',')

        read_names = df['QUERY'].unique()
        print(name, ' candidates num:', len(read_names))
        num = 0
        for r_name in read_names:
            df_tmp = df[df['QUERY'] == r_name].copy(deep=True)
            # uniaue match
            if len(df_tmp['INS_COORD_B'].unique()) == 1:
                # 1 repeat or no repeat
                if df_tmp.shape[0] == 1:
                    if is_questionable(df_tmp.iloc[0], is_line):
                        df.drop(df_tmp.index, axis=0, inplace=True)
                        num += 1
                        #print(r_name)
                # 2 repeats in the area of the ins
                else:
                    for i, row in df_tmp.iterrows():
                        if is_questionable(row, is_line):
                            df.drop(df_tmp.index, axis=0, inplace=True)
                            num += 1
                            #print(r_name)
                            break
            # not unique => drop
            else:
                df.drop(df_tmp.index, axis=0, inplace=True)
                num += 1
                #print(r_name)
        print(name, ' dropped:', num)
        df.to_csv(DIR + '/result/' + f'{name}_result.csv', index=False)

In [8]:
DIR = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/AGCT/'
os.chdir(DIR)

dirs = os.listdir()
for dirname in dirs:
    print(dirname, '---------------------------------------------------')
    curr_dir = DIR + dirname 
    if not os.path.isdir(curr_dir):
        continue
    os.chdir(curr_dir)
    
    if not os.path.exists(curr_dir + '/nofiltration/result'):
        os.makedirs(curr_dir + '/nofiltration/result')
    
    # 1 Get somatic insertions
    file_names, df_somatic_list = get_somatic(curr_dir)
    # drop
    drop_questionable(curr_dir + '/nofiltration', file_names, is_line=True)

BC60 ---------------------------------------------------
metatable shape: (1907, 32)
sample file names: ['BC60-L_S106_ME_L001__', 'MN-BC60-L_S108_ME_L001__']
null occurrences: 0
all not somatic: 828   including test: 0
somatic and quasi-somatic: (1079, 32)   including test: 0 

BC60-L_S106_ME_L001__ 441   test: 0
MN-BC60-L_S108_ME_L001__ 638   test: 0
overall num of somatic insertions:  1079 

BC60-L_S106_ME_L001__  candidates num: 23
BC60-L_S106_ME_L001__  dropped: 4
MN-BC60-L_S108_ME_L001__  candidates num: 35
MN-BC60-L_S108_ME_L001__  dropped: 8
BC66 ---------------------------------------------------
metatable shape: (1927, 32)
sample file names: ['BC66-L_S102_ME_L001__', 'MN-BC66-L_S104_ME_L001__']
null occurrences: 0
all not somatic: 842   including test: 0
somatic and quasi-somatic: (1085, 32)   including test: 0 

BC66-L_S102_ME_L001__ 382   test: 0
MN-BC66-L_S104_ME_L001__ 703   test: 0
overall num of somatic insertions:  1085 

BC66-L_S102_ME_L001__  candidates num: 21
BC66-L

### Tumor norm matches 

In [12]:
# вставки в раке Алю1 - вставки в норме Рса; 
# Вставки в раке Рса - вставки в норме Алю1, для того же пациента
def check_matches_tumor_mn(curr_dir1, curr_dir2, file_names, dirname, filter, del_matches):

    window = 20
    num_matched = [dirname]

    if file_names[0].find('MN-') != -1:
        tmp = str(file_names[1])
        file_names[1] = str(file_names[0])
        file_names[0] = str(tmp)
    # 0 - tumor, 1 - matched norm
    
    filtration = ''
    if not filter:
        filtration = '/nofiltration'
    #
    df_tumor_Alu1 = pd.read_table(curr_dir1 + filtration + '/result/' + f'{file_names[0]}_result.csv', sep=',')
    df_tumor_Rsa1 = pd.read_table(curr_dir2 + filtration + '/result/' + f'{file_names[0]}_result.csv', sep=',')
    df_norm_Alu1 = pd.read_table(curr_dir1 + filtration + '/result/' + f'{file_names[1]}_result.csv', sep=',')
    df_norm_Rsa1 = pd.read_table(curr_dir2 + filtration + '/result/' + f'{file_names[1]}_result.csv', sep=',')

    df_t_Alu1_mn_Rsa1 = pd.DataFrame(columns=df_tumor_Alu1.columns)
    for i, row1 in df_tumor_Alu1.iterrows():
        for j, row2 in df_norm_Rsa1.iterrows():
            if (row1['INS_CHR_B'] == row2['INS_CHR_B']) and (abs(row1['INS_COORD_B'] - row2['INS_COORD_B']) <= window):
                if del_matches:
                    df_tumor_Alu1.drop(i, axis=0, inplace=True)
                    df_norm_Rsa1.drop(j, axis=0, inplace=True)
                df_t_Alu1_mn_Rsa1.loc[len(df_t_Alu1_mn_Rsa1.index)] = row1

    df_t_Rsa1_mn_Alu1 = pd.DataFrame(columns=df_tumor_Alu1.columns)
    for i, row1 in df_tumor_Rsa1.iterrows():
        for j, row2 in df_norm_Alu1.iterrows():
            if (row1['INS_CHR_B'] == row2['INS_CHR_B']) and (abs(row1['INS_COORD_B'] - row2['INS_COORD_B']) <= window):
                if del_matches:
                    df_tumor_Rsa1.drop(i, axis=0, inplace=True)
                    df_norm_Alu1.drop(j, axis=0, inplace=True)
                df_t_Rsa1_mn_Alu1.loc[len(df_t_Rsa1_mn_Alu1.index)] = row1

    if del_matches:
        df_tumor_Alu1.to_csv(curr_dir1 + filtration + '/result/' + f'{file_names[0]}_result.csv', index=False)
        df_tumor_Rsa1.to_csv(curr_dir2 + filtration + '/result/' + f'{file_names[0]}_result.csv', index=False)
        df_norm_Alu1.to_csv(curr_dir1 + filtration + '/result/' + f'{file_names[1]}_result.csv', index=False)
        df_norm_Rsa1.to_csv(curr_dir2 + filtration + '/result/' + f'{file_names[1]}_result.csv', index=False)
    
    num_matched.append(len(df_t_Alu1_mn_Rsa1['QUERY'].unique()))
    num_matched.append(len(df_t_Rsa1_mn_Alu1['QUERY'].unique()))
    num_matched.append(len(df_tumor_Alu1['QUERY'].unique()))
    num_matched.append(len(df_tumor_Rsa1['QUERY'].unique()))
    num_matched.append(len(df_norm_Alu1['QUERY'].unique()))
    num_matched.append(len(df_norm_Rsa1['QUERY'].unique()))
    
    return num_matched

In [11]:
DIR1 = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/AGCT/'
DIR2 = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/GGCC/'
SAVE_TO = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/'
FILTER = False
DEL_MATCHES = True
LINE = True

dirs = os.listdir(DIR1)
if LINE:
    num_matched = pd.DataFrame(columns=['NAME', 'TUMOR_Alu1_MN_HaeIII', 'TUMOR_HaeIII_MN_Alu1', 'TUMOR_Alu1', 'TUMOR_HaeIII', 'MN_Alu1', 'MN_HaeIII'])
else:
    num_matched = pd.DataFrame(columns=['NAME', 'TUMOR_Alu1_MN_Rsa1', 'TUMOR_Rsa1_MN_Alu1', 'TUMOR_Alu1', 'TUMOR_Rsa1', 'MN_Alu1', 'MN_Rsa1'])
for dirname in dirs:

    curr_dir1 = DIR1 + dirname
    curr_dir2 = DIR2 + dirname
    
    if not os.path.isdir(curr_dir1):
        continue
    
    # 1 Get somatic insertions
    file_names, df_somatic_list = get_somatic(curr_dir1, show=False)
    # tumor norm matches 
    num_matched.loc[len(num_matched.index)] = check_matches_tumor_mn(curr_dir1, curr_dir2, file_names, dirname, FILTER, DEL_MATCHES)
    
if FILTER:
    num_matched.to_csv(SAVE_TO + f'matches_tumor_mn_filtr.csv', index=False)
else:
    num_matched.to_csv(SAVE_TO + f'matches_tumor_mn_nofiltr.csv', index=False)

num_matched

,NAME,TUMOR_Alu1_MN_HaeIII,TUMOR_HaeIII_MN_Alu1,TUMOR_Alu1,TUMOR_HaeIII,MN_Alu1,MN_HaeIII
0,BC60,0,0,19,12,27,18
1,BC66,0,0,17,18,24,12
2,BC70,0,0,5,3,30,20
3,BLC1,0,0,20,10,7,9
4,CC107,0,0,19,11,11,3
5,CC127-2,0,0,51,38,14,6
6,CC18,0,0,13,10,37,34
7,CC46,0,0,7,3,21,9
8,CC5,0,0,12,22,23,8
9,CC82,0,0,47,34,15,2


### Get values

In [13]:
def get_coords(curr_dir1, curr_dir2, file_names, save_to, filter, is_LINE):

    filtration = ''
    if not filter:
        filtration = '/nofiltration'

    for name in file_names:
        df = pd.read_table(curr_dir1 + filtration + '/result/' + f'{name}_result.csv', sep=',')
        coord_df1 = df[['QUERY', 'INS_CHR_B', 'INS_COORD_B', 'INS_STRAND_B', f'{name}_NUM_READS']].copy(deep=True)
        coord_df1.columns = ['READ_NAME', 'INS_CHR', 'INS_COORD', 'INS_STRAND', 'NUM_READS']
        coord_df1['RESTRICTASE'] = 'Alu1'
        
        df = pd.read_table(curr_dir2 + filtration + '/result/' + f'{name}_result.csv', sep=',')
        coord_df2 = df[['QUERY', 'INS_CHR_B', 'INS_COORD_B', 'INS_STRAND_B', f'{name}_NUM_READS']].copy(deep=True)
        coord_df2.columns = ['READ_NAME', 'INS_CHR', 'INS_COORD', 'INS_STRAND', 'NUM_READS']
        if is_LINE:
            coord_df2['RESTRICTASE'] = 'HaeIII'
        else:
            coord_df2['RESTRICTASE'] = 'Rsa1'

        for i, row1 in coord_df1.iterrows():
            for j, row2 in coord_df2.iterrows():
                if (row1['INS_CHR'] == row2['INS_CHR']) and (abs(row1['INS_COORD'] - row2['INS_COORD']) <= 10):
                    if is_LINE:
                        coord_df1.at[i, 'RESTRICTASE'] = 'Alu1, HaeIII'
                    else:
                        coord_df1.at[i, 'RESTRICTASE'] = 'Alu1, Rsa1'
                    coord_df1.at[i, 'NUM_READS'] = row1['NUM_READS'] + row2['NUM_READS']
                    coord_df2.drop(j, axis=0, inplace=True)
        
        df_to_save = pd.concat([coord_df1, coord_df2], ignore_index=True).drop_duplicates()
        
        df_to_save.to_csv(save_to + f'{name}_coords.csv', index=False)

In [14]:
def check_matched_coords(curr_dir1, curr_dir2, file_names, dirname, filter):

    num_matched = [dirname]

    filtration = ''
    if not filter:
        filtration = '/nofiltration'

    if file_names[0].find('MN-') != -1:
        tmp = str(file_names[1])
        file_names[1] = str(file_names[0])
        file_names[0] = str(tmp)
    
    for name in file_names:
        
        df1 = pd.read_table(curr_dir1 + filtration + '/result/' + f'{name}_result.csv', sep=',')
        df2 = pd.read_table(curr_dir2 + filtration + '/result/' + f'{name}_result.csv', sep=',')

        df_to_save = pd.DataFrame(columns=df2.columns)
        
        for i, row1 in df1.iterrows():
            for j, row2 in df2.iterrows():
                if (row1['INS_CHR_B'] == row2['INS_CHR_B']) and (abs(row1['INS_COORD_B'] - row2['INS_COORD_B']) <= 10):
                    df_to_save.loc[len(df_to_save.index)] = row1

        num_matched.append(len(df_to_save['QUERY'].unique()))
        num_matched.append(len(df1['QUERY'].unique()))
        num_matched.append(len(df2['QUERY'].unique()))
    
    return num_matched

In [15]:
DIR1 = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/AGCT/'
DIR2 = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/GGCC/'
SAVE_TO = '/mnt/c/Users/Komar/Lab/OutPutDir/LINE/'
filter = False
LINE = True

dirs = os.listdir(DIR1)
if LINE:
    num_matched = pd.DataFrame(columns=['NAME', 'MATCHED', 'Alu1', 'HaeIII', 'MN_MATCHED', 'MN_Alu1', 'MN_HaeIII'])
else:
    num_matched = pd.DataFrame(columns=['NAME', 'MATCHED', 'Alu1', 'Rsa1', 'MN_MATCHED', 'MN_Alu1', 'MN_Rsa1'])
general_ins_num = pd.DataFrame(columns=['NAME', 'TOTAL_INS', 'MN_TOTAL'])
for dirname in dirs:

    curr_dir1 = DIR1 + dirname
    curr_dir2 = DIR2 + dirname
    
    if not os.path.isdir(curr_dir1):
        continue
    
    # 1 Get somatic insertions
    file_names, df_somatic_list = get_somatic(curr_dir1, show=False)
    # get coords for Maria
    get_coords(curr_dir1, curr_dir2, file_names, SAVE_TO, filter, LINE)
    # check
    matches = check_matched_coords(curr_dir1, curr_dir2, file_names, dirname, filter)
    num_matched.loc[len(num_matched.index)] = matches
    general_ins_num.loc[len(general_ins_num.index)] = [matches[0], matches[2] + matches[3] - matches[1],
                                                       matches[5] + matches[6] - matches[4],
                                                      ]

if filter:
    num_matched.to_csv(SAVE_TO + f'matched_ins_filtr.csv', index=False)
    general_ins_num.to_csv(SAVE_TO + f'total_ins_filtr.csv', index=False)
else:
    num_matched.to_csv(SAVE_TO + f'matched_ins_nofiltr.csv', index=False)
    general_ins_num.to_csv(SAVE_TO + f'total_ins_nofiltr.csv', index=False)

num_matched

,NAME,MATCHED,Alu1,HaeIII,MN_MATCHED,MN_Alu1,MN_HaeIII
0,BC60,1,19,12,5,27,18
1,BC66,1,17,18,1,24,12
2,BC70,1,5,3,3,30,20
3,BLC1,1,20,10,0,7,9
4,CC107,2,19,11,0,11,3
5,CC127-2,8,51,38,0,14,6
6,CC18,2,13,10,9,37,34
7,CC46,0,7,3,2,21,9
8,CC5,3,12,22,2,23,8
9,CC82,9,47,34,0,15,2


In [16]:
general_ins_num

,NAME,TOTAL_INS,MN_TOTAL
0,BC60,30,40
1,BC66,34,35
2,BC70,7,47
3,BLC1,29,16
4,CC107,28,14
5,CC127-2,81,20
6,CC18,21,62
7,CC46,10,28
8,CC5,31,29
9,CC82,72,17
